In [ ]:
# https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/
# (1) from command line ----    git clone https://github.com/matterport/Mask_RCNN.git
#      after clone --- command line --- cd Mask_RCNN
#                              --- (sudo) python setup.py install

# (2) from command line --- get dataset --- git clone https://github.com/experiencor/kangaroo.git
# (3) download the model (below)
    # pip install wget
    #import wget
    #url = 'https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5'
    #wget.download(url, 'mask_rcnn_coco.h5')
     
# (4) pip install tensorflow-gpu==1.14  *important for this version and use GPU


# (5) # pip install keras==2.2.4    #important version
# (6) # # pip install mrcnn
# train a new object, for example, Kangaroo or something else 

import warnings
warnings.simplefilter("ignore")
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os

In [ ]:
epochs = 100
STEPS_PER_EPOCH = 783



class ClownDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		self.add_class("dataset", 2, "nface")  
		self.add_class("dataset", 3, "color")
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		for filename in listdir(images_dir):
			image_id = filename[:-4]
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids=[0,1,2,3]) #mod**

	def extract_boxes(self, filename):
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if i == 0:                                    
				masks[row_s:row_e, col_s:col_e, i] = 1                 #mod  
				class_ids.append(self.class_names.index('clown'))      #mod
			elif i == 1:                                         
				masks[row_s:row_e, col_s:col_e, i] = 2                 #mod
				class_ids.append(self.class_names.index('nface'))     #mod
			elif i == 2:   
				masks[row_s:row_e, col_s:col_e, i] = 3                 #mod
				class_ids.append(self.class_names.index('color'))     #mod
		return masks, asarray(class_ids, dtype='int32')                #mod

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']


class ClownConfig(Config):
	NAME = "Clown_cfg"
	NUM_CLASSES = 1 + 3
	STEPS_PER_EPOCH = STEPS_PER_EPOCH


train_set = ClownDataset()
train_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/training', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
test_set = ClownDataset()
test_set.load_dataset('/home/ubuntu/gitscripts/FobiaPhilter/validation', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
config = ClownConfig()
config.display()

model = MaskRCNN(mode='training', model_dir='./', config=config)
model.load_weights('/home/ubuntu/gitscripts/FobiaPhilter/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=epochs, layers='heads')